# KELLOGGS MATRIX
- author: Richard Castro 
- sticky_rank: 4
- toc: false
- badges: false
- comments: false
- categories: [Matrix]
- image: images/kelloggs.png

In [ ]:
#collapse-hide
#IMPORT LIBRARIES FROM PYTHON
import pandas as pd
pd.options.display.max_rows = 2800

In [ ]:
#collapse-hide
#IMPORT DATA SETS FROM TRUSTED SOURCES

#US CENSUS DATA FROM -
censusData = pd.read_csv('../_data/cen.csv')
censusData.columns=map(str.lower, censusData.columns)
#COUNTY MASK USAGE FROM -
maskData = pd.read_csv('../_data/maskbycounty.csv')
maskData.columns=map(str.lower, maskData.columns)

#CLIENT LOCATION DATA
locations=pd.read_csv('../_data/kellogg_locations.csv')
locations
loc=['state', 'county']
showLocs=locations[loc]
showLocs.set_index('state', inplace=True)
print('These are the US Locations for: Kelloggs')
showLocs

In [ ]:
#collapse-hide
#IMPORT LIBRARIES
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap

cities = pd.read_csv('../_data/client_locations/kelloggs.csv')
lat = cities['lat'].values
lon = cities['long_'].values
population = cities['active'].values
area = cities['tee'].values
#collapse-hide
fig = plt.figure(figsize=(18, 10))

m = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc', resolution='l', lat_1=33,lat_2=45,lon_0=-95)

m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
m.drawcounties(color='gray')


m.scatter(lon, lat, latlon=True,
          c=np.log10(population), s=area,
          cmap='Reds', alpha=1)


plt.title("Kelloggs Locations")
plt.colorbar(label=r'Covid Cases(Active)')
plt.clim(3, 7)

In [ ]:
#collapse-hide
#RTLIVE DATA
rt=pd.read_csv('../_data/rt.csv')
#TN DATA IMPORT
tn=pd.read_csv('../_data/tn.csv')
tn.columns=map(str.lower, tn.columns)
#OH DATA IMPORT
oh=pd.read_csv('../_data/ohio.csv')
oh.columns=map(str.lower, oh.columns)
#FL DATA IMPORT
fl=pd.read_csv('../_data/fl.csv')
fl.columns=map(str.lower, fl.columns)
#RT DATA CLEANING
rt=rt.rename(columns={'region':'st'})
rtC=['date', 'st', 'mean']
rta=rt[rtC]

#CLIENT AND RT DATA  MERGE
LocationRT=pd.merge(locations, rta, on='st')
Location=LocationRT[LocationRT['date']=='8/2/2020']
#CENSUS DATA CLEANING
cenC=['fips','popestimate2019']
cen=censusData[cenC]
#CLIENT AND CENSUS DATA MERGE
locationCen=pd.merge(Location.astype(str), cen, on='fips')
locationMask=pd.merge(locationCen, maskData.astype(str), on='fips')
matrixReport=locationMask
locations

In [ ]:
#collapse-hide
tn=tn.rename(columns={'state':'st', 'test_pos':'testP', 'test_neg':'testN', 'test_tot':'testT',
       'test_new':'testNew', 'cases_confirmed':'cases', 'new_cases_confirmed':'casesNewCf', 'cases_probable':'casesP',
       'cases_new_probable':'casesNP', 'cases_tot':'caseT', 'cases_new':'caseNew', 'hospitalized_tot':'hospT',
       'hospitalized_new':'hospN', 'recov_tot':'recT', 'recov_new':'recN', 'deaths_tot':'deathsT',
       'deaths_new':'deathsNew', 'active_tot':'actT', 'active_new':'actNew'})

tnCol=['date', 'st', 'county', 'testP', 'testN', 'testT','testNew', 'cases', 'casesNewCf','casesP',
       'casesNP', 'caseT', 'caseNew', 'hospT','hospN', 'recT', 'recN', 'deathsT',
       'deathsNew', 'actT', 'actNew']
tnc=tn[tnCol]
tnc

In [ ]:
#collapse-hide
fl=fl.rename(columns={'cases_c':'caseT', 'test_t':'testT', 'test_n':'testN', 'test_p':'testP',
       'deaths_tot':'deathsT'})
flCol=['county', 'state', 'caseT', 'testT', 'testN', 'testP',
       'deathsT']
flc=fl[flCol]

In [ ]:
#collapse-hide
ohCol=['case count', 'death count', 'hospitalized count',
       'fips']
ohC=oh[ohCol]
ohC=ohC.rename(columns={'case count':'caseT', 'death count':'deathT', 'hospitalized count':'hospT'})

In [ ]:
#collapse-hide
matrixMix=pd.merge(matrixReport, ohC.astype(str), on='fips', how='left')
flc=flc.rename(columns={'state':'st'})
matrixM=pd.merge(matrixMix, flc, on=['county', 'st'], how='left')
matrixM.head(1)

In [ ]:
#collapse-hide
tnc.columns

In [ ]:
#collapse-hide
mat=pd.merge(matrixM.astype(str)[pd.notnull], tnc.astype(str)[pd.notnull], on=['st', 'county',  'deathsT', 'testT', 'testN', 'hospT'], how='left')
pd.options.display.max_rows = 2800
pd.options.display.max_columns = 2800
mat

In [ ]:
#collapse-hide
#RT DATA FROM -
D=pd.read_csv('../_data/rttestcounty.csv')
countyData = pd.read_csv('../_data/isee.csv')
rtData = pd.read_csv('https://d14wlfuexuxgcm.cloudfront.net/covid/rt.csv')

#CLEAN UP DATA AND STANDARDIZE THE COLUMNS
ACol=[ 'uid', 'date', 'resolution', 'date_lag', 'Rt_plot', 'Rt_upr',
       'Rt_lwr', 'Rt_loess_fit', 'Rt_loess_lwr', 'Rt_loess_upr',
       'positiveIncrease', 'positive', 'positive_7day', 'positive_percapita',
       'positiveIncr_percapita', 'deathIncrease', 'death', 'death_percapita',
       'deathIncr_percapita']
As=countyData[ACol]

pCol=['fips', 'POPESTIMATE2019']
censusDataCleaned=censusData[pCol]

#KELLOGGS LOCATIONS
clientLocations=pd.read_csv('../_data/kellogg_locations.csv')
fCol=['uid', 'fips', 'state', 'county']
clientDataCleaned=clientLocations[fCol]
clientDataCleaned


mergeClientCensus=pd.merge(censusDataCleaned, clientDataCleaned.astype(str), on ="fips")
mergeData=mergeClientCensus.sort_values('state')


md=pd.merge(mergeData, D.astype(str), on='fips')
mn=md.sort_values('state')

UT=pd.merge(mergeData, As.astype(str), on='uid')
today = '08/01/2020'
UTT = UT[UT['date']==today]

AwA=pd.merge(As.astype(str), mergeData, on='uid', )

AwCol=[ 'state', 'county', 'date', 'resolution', 'Rt_plot', 'Rt_upr', 'Rt_lwr',
       'Rt_loess_fit', 'Rt_loess_lwr', 'Rt_loess_upr', 'positiveIncrease',
       'positive', 'positive_7day', 'positive_percapita',
       'positiveIncr_percapita', 'deathIncrease', 'death', 'death_percapita',
       'deathIncr_percapita',  'POPESTIMATE2019', 'fips' ]
AwCleaned = AwA[AwCol]

ATT=pd.merge(AwCleaned, maskData.astype(str), on='fips')
pd.options.display.max_rows = 2800

ATCol=['state', 'county', 'date', 'Rt_plot', 'positiveIncrease',
       'positive', 'positive_7day', 'positive_percapita',
       'positiveIncr_percapita', 'deathIncrease', 'death', 'death_percapita',
       'deathIncr_percapita', 'POPESTIMATE2019', 'NEVER', 'RARELY',
       'SOMETIMES', 'FREQUENTLY', 'ALWAYS']
ATR=ATT[ATCol]
ATY = ATR[ATR['date']=='7/18/2020']